In [1]:
from snake_settings import Settings
import snake_game_functions as gf
import snake_agent as ag

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from torch.utils.tensorboard import SummaryWriter

settings = Settings()
snake,apple = gf.new_game(settings)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_net, target_net, optimizer, memory = ag.create_agent(settings,device,ag.MEMORY_SIZE)
writer = SummaryWriter()

In [2]:
#Optional loading weights of NN from file

print('Load NN weights from file?\ny or n')
choice = input()
if choice =='y':
    checkpoint = torch.load('../input/snake-weights/w_100f_512n_512n')
    policy_net.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [11]:
print('Choose epoch number\nnumber')
choice = input()

In [12]:
%%time
run=True
epoch=0
while run:
    if choice == str(epoch):
        run = False
    epoch+=1
    state = ag.get_screen(settings, snake, apple,device)
    action = ag.select_action(state,policy_net,device)
    if run:
        gf.check_events(settings,snake,apple,action)
        reward = torch.tensor([snake.reward],device=device)
        if snake.dead:
            writer.add_scalar('Length',len(snake.body),epoch)
            snake,apple = gf.new_game(settings)
        next_state = ag.get_screen(settings, snake, apple, device)
        memory.push(state, action, next_state, reward)
        loss1 = ag.optimize_model(memory,device,policy_net,target_net,optimizer,snake)
        if loss1 != None:
            writer.add_scalar('Loss',loss1,epoch)

In [ ]:
print('Save NN weights in file?\n y or n')
choice = input()

if choice =='y':
    torch.save({
                'model_state_dict': policy_net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),

                }, 'w_100f_512n_512n')